In [ ]:
%load_ext autoreload
%aimport rasterio_helpers
%aimport proj_scope_vars
%autoreload 1
%autosave 300

import importlib 

import pdal
import glob
import json
import os
from rasterio_helpers import *
from proj_scope_vars import *
import rasterio
from rasterio.warp import reproject
import numpy as np
import rasterio as rio
from rasterio.features import dataset_features
import geopandas as gpd


In [ ]:
def mask_merge(tiffs, outtiff=None):
    masked_dss = []
    for tiff in tiffs:
        with rasterio.open(tiff) as ds:
            masked_dss.append(mask_ds(ds, height_map_mask))
    
    mask_merge_ds = merge_ds(masked_dss)
    if outtiff is not None:
        save_ds(mask_merge_ds, outtiff)
    return mask_merge_ds

In [ ]:
#Intraband Alignment for skysat data. 
# This masks to lidar area, then alings masked ms data to pan chromatic and multispectral data.
# Finally, this puts everything back into a single file.
panband_tiffs = glob.glob(f'{skysat_dl_dir}/*panchromatic*.tif')
ms_tiffs = glob.glob(f'{skysat_dl_dir}/*pansharpened*.tif')

# sks_aligned_tiff = f'{aligned_raster_dir}/{skysat_basename}_alinged.tif'

pan_masked_tiff =  f'{skysat_processed_dir}/{skysat_basename}_pan_merged_masked.tif'
ms_masked_tiff =  f'{skysat_processed_dir}/{skysat_basename}_ms_merged_masked.tif'

pan_masked_ds = mask_merge(panband_tiffs, pan_masked_tiff)
ms_masked_ds  = mask_merge(ms_tiffs, ms_masked_tiff)

In [ ]:


sks_idxs = list(skysat_dl_bands.keys())
sks_idxs.sort()

ms_band_profile = ms_masked_ds.profile
ms_band_profile.update( {"count": 1})

# for idx in sks_idxs:
#     bandname = skysat_dl_bands[idx]
#     ms_band_masked_tiff  = f'{temp_dir}/{skysat_basename}_{bandname}_masked.tif'
#     ms_band_aligned_tiff = f'{temp_dir}/{skysat_basename}_{bandname}_aligned.tif'
#     work_dir = f'{skysat_dl_dir}/work/{skysat_basename}_{bandname}_iba
    
#     with rasterio.open(ms_band_masked_tiff, 'w', **ms_band_profile) as ms_masked_band_ds:
#         ms_band_ds.write(1, ms_masked_ds.read(idx))
#         ms_band_ds.set_band_description(1, bandname)
   
#     align_rasters(pan_masked_tiff, 
#                   ms_band_masked_tiff, 
#                   ms_band_aligned_tiff, 
#                   work_dir=work_dir
#                   ref_band=None, tgt_band=None)
    
#     with rasterio.open(ms_band_aligned_tiff) as ms_band_aligned_ds:
#         band_alinged_dss.append(ms_band_aligned_ds)

#     os.remove(ms_band_masked_tiff)
#     os.remove(ms_band_aligned_tiff)

# os.remove(pan_masked_tiff)
    
# band_alinged_dss.append(pan_masked_ds)

# aligned_profile = ms_band_profile
# aligned_profile.update({"count": len(band_alinged_dss)})

# with rasterio.open(sks_aligned_tiff, 'w', **aligned_profile) as aligned_ds:
#     for idx in range(len(band_alinged_dss)):
#         band = idx+1
#         bandname = skysat_dl_bands[band]
#         band_alinged_ds = band_alinged_dss[idx]
#         aligned_ds.write(band_idx, band_alinged_ds.read(band))
#         aligned_ds.set_band_description(band, bandname)


In [ ]:
skysat_sf_tiff = 
with rasterio.open('stack.tif', 'w', **meta) as dst:
    

In [ ]:
sks_sf_tiffs    = glob.glob(skysat_dl_dir + '/*.tif')
print(sks_sf_tiffs)

if not os.path.exists(skysat_processed_dir):
    os.makedirs(skysat_processed_dir)

for sks_sf_tiff in sks_sf_tiffs:
    file_basename = os.path.splitext(os.path.basename(sks_sf_tiff))[0]
    sks_sf_outfn = f'{skysat_processed_dir}/{file_basename}_masked.tif'
    
    print(sks_sf_tiff)
    try:
        sks_sf_masked_ds  = mask_ds(sks_sf_ds, 
                                   height_map_mask, 
                                   os.path.splitext(os.path.basename(height_map_mask))[0], 
                                   nodata=0.0)
    except ValueError:
        continue
    save_ds(sks_sf_masked_ds, sks_sf_outfn)

In [ ]:
sks_sf_tiffs     = glob.glob(skysat_processed_dir + '/*masked.tif')
sks_sf_outfn     = f'{skysat_processed_dir}/{skysat_basename}_merged.tif'
sks_sf_ds_list   = [rasterio.open(sks_sf_tiff) for sks_sf_tiff in sks_sf_tiffs]
sks_sf_merged_ds = merge_ds(sks_sf_ds_list)
save_ds(sks_sf_masked_ds, sks_sf_outfn)